In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import numpy as np 
import torch
import torch.nn.functional as F
import torchvision
from datasets import load_dataset
from diffusers import DDPMPipeline, DDIMScheduler
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
device

c:\Users\Triton Neo 16\diffusion-teset\venv\lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


'cuda'

In [3]:
pipeline_name = "johnowhitaker/sd-class-wikiart-from-bedrooms"
image_pipe = DDPMPipeline.from_pretrained(pipeline_name)

scheduler = DDIMScheduler.from_pretrained(pipeline_name)
scheduler.set_timesteps(num_inference_steps=40)

x = torch.randn(8, 3, 256, 256).to(device)
for i, t in tqdm(enumerate(scheduler.timesteps), total=len(scheduler.timesteps)):
    model_input = scheduler.scale_model_input()
    x = image_pipe(x, timesteps=t)
    if i % 10 == 0:
        plt.imshow(x[0].cpu().detach().permute(1, 2, 0))
        plt.show(


TypeError: __init__() missing 2 required positional arguments: 'unet' and 'scheduler'

In [1]:
def color_loss(images, target_color=(.1, .9, .5)):
    target = (
        torch.tensor(target_color).to(images.device) * 2 - 1
    )
    target = target[
        None, :, None, None
    ]
    error = torch.abs(
        images - target
    ).mean()
    return error